The goal of this project is to target essential features in each load in order to more accurately pair them with receiving loads.
To get the exact truck which each transfer load went to is nearly impossible without using brute force, so they must be based on the key
features such as:
     1. Province.
     2. Weight.
     3. Volume.
     4. Market.

Even a hand count by a human operator would result in too many errors, and not be able to pair each load with its recipient--therefore,
the best that can be done is to save the expense of human users of going through potentially thousands of loads, when their time could be
utilized elsewhere.

In [1]:
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gspread as gs
import ezodf
import openpyxl
from pandas_ods_reader import read_ods
from collections import Counter
import ezodf
import datetime as dt
import xlsxwriter
import calendar


In [2]:
# filename = pd.ExcelFile("January 2023 Dispatch Sheets.ods")
filename = "TMW_DATA"

In [17]:
df = pd.read_excel(filename+".xlsX")




###########FILTER OUT IRRELEVANT COLUMNS#########

#df = df.drop(['Unit','Trl1','Drv','Last Inv','Pay Check','Remarks','Unnamed: 22','Unnamed: 23','Carrier','Service','BOL','Product Name',
 #             'Loading Point','Destination'],axis=1)


In [18]:
##########DATA CLEANING/STRUCTURING##############

df['Started'] = pd.to_datetime(df['Started']) # convert started column to date time series
df['Completed'] = pd.to_datetime(df['Completed']) # convert started column to date time series

###For consideration for a future GUI, these ranges could be filtered.
df = df[df['Started'].dt.year & df['Completed'].dt.year > 2022] #date cleaning. 
df = df[df['Started'].dt.month & df['Completed'].dt.month > 3] #date cleaning

df['Started'] = df['Started'].dt.month #Only get the months now.
df['Completed'] = df['Completed'].dt.month #Only get the months now.


d = dict(enumerate(calendar.month_abbr))

df['Started'] = df['Started'].map(d)
df['Completed'] = df['Completed'].map(d)


#######################Here, weight and volume is rounded to the nearest 1000th, since not all drivers accurately enter data on tablet#####

df.Weight = df["Weight"].round(decimals=-3)
df.Volume = df["Volume"].round(decimals=-3)

df['Shared Date'] = " "



In [25]:
df

,Unnamed: 0,TMW#,Status,Started,Completed,Load,Dest.,Prod.,Volume,Weight,Market,Shared Date
4544,4544,132684,CMP,Apr,Apr,TIGMIT,TIGNOR01,1360X00,30000.0,40000.0,DUST,
4545,4545,132680,CMP,Apr,Apr,TIGMIT,TIGSPI,1360X00,27000.0,37000.0,DUST,
4548,4548,132687,CMP,Apr,Apr,TIGMIT,TIGNOR01,1360X00,29000.0,40000.0,DUST,
4549,4549,132688,CMP,Apr,Apr,TIGMIT,CUSTPICK,1200X11,24000.0,30000.0,OIL,
4550,4550,132681,CMP,Apr,Apr,TIGMIT,CUSTPICK,1360X17,28000.0,38000.0,OIL,
...,...,...,...,...,...,...,...,...,...,...,...,...
8442,8442,137214,CMP,Jun,Jun,TIGPRI05,GOVPRI03,1360X00,29000.0,39000.0,DUST,
8443,8443,137177,CMP,Jun,Jun,TIGVAN01,BABBUR02,1360X00,30000.0,41000.0,DUST,
8444,8444,137195,CMP,Jun,Jun,TIGMIT,PARFOX11,1200X11,28000.0,34000.0,OIL,
8445,8445,137268,CAN,Jun,Jun,TIGMIT,DAWFOR,1360X00,0.0,0.0,DUST,


In [39]:
######EACH PROVINCE HAS THEIR OWN RESPECTIVE LOAD AND DESTINATION DATAFRAMES###########

ttbc_load = df[df['Load'] == 'TTBC']
ttbc_dest = df[df['Dest.'] == 'TTBC']

ttsk_load = df[df['Load'] == 'TTSK']
ttsk_dest = df[df['Dest.'] == 'TTSK']


ttab_load = df[df['Load'] == 'TTAB']
ttab_dest = df[df['Dest.'] == 'TTAB']

#df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})

########CREATE A SHARED DATE COLUMN FOR START/COMPLETE##########

ttbc_load.loc[:,'Shared Date'] = ttbc_load['Completed']
ttbc_dest.loc[:,'Shared Date'] = ttbc_dest['Started'] 

ttab_load.loc[:,'Shared Date'] = ttab_load['Completed']
ttab_dest.loc[:,'Shared Date'] = ttab_dest['Started'] 

ttsk_load.loc[:,'Shared Date'] = ttsk_load['Completed']
ttsk_dest.loc[:,'Shared Date'] = ttsk_dest['Started'] 


/var/folders/79/y1kk4mzj387ckxj5f0f4hsjw0000gn/T/ipykernel_6542/4056206000.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ttbc_load.loc[:,'Shared Date'] = ttbc_load['Completed']
/var/folders/79/y1kk4mzj387ckxj5f0f4hsjw0000gn/T/ipykernel_6542/4056206000.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ttbc_dest.loc[:,'Shared Date'] = ttbc_dest['Started']
/var/folders/79/y1kk4mzj387ckxj5f0f4hsjw0000gn/T/ipykernel_6542/4056206000.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy o

Now that a shared date column has been created, the loads are more unique and can be split up accordingly.

In [40]:
key_cols = ['Prod.', 'Weight', 'Volume','Market','Shared Date'] #Key columns incorporated into pairing process
#ab_merge = ttab_load.merge(ttab_load.loc[:, ttab_dest.columns.isin(key_cols)])

ab_merge = pd.merge(ttab_load, ttab_dest, on = key_cols)
ab_merge = ab_merge.rename(columns={'TMW#_x': 'LOAD TMW#'})
ab_merge = ab_merge.rename(columns={'TMW#_y': 'DEST TMW#'})

bc_merge = pd.merge(ttbc_load, ttbc_dest, on = key_cols)
bc_merge = bc_merge.rename(columns={'TMW#_x': 'LOAD TMW#'})
bc_merge = bc_merge.rename(columns={'TMW#_y': 'DEST TMW#'})

sk_merge = pd.merge(ttsk_load, ttsk_dest, on = key_cols)
sk_merge = sk_merge.rename(columns={'TMW#_x': 'LOAD TMW#'})
sk_merge = sk_merge.rename(columns={'TMW#_y': 'DEST TMW#'})

ab_unused = pd.merge(ttab_load, ttab_dest, how='outer', on=key_cols, indicator=True)
sk_unused = pd.merge(ttsk_load, ttsk_dest, how='outer', on=key_cols, indicator=True)
bc_unused = pd.merge(ttbc_load, ttbc_dest, how='outer', on=key_cols, indicator=True)

ab_unused = ab_unused.drop(['Dest._x','Dest._y','Status_x','Status_y','Market','Prod.','Shared Date'],axis=1)
ab_unused = ab_unused.rename(columns={'_merge': 'UNIQUE'})

sk_unused = sk_unused.drop(['Dest._x','Dest._y','Status_x','Status_y','Market','Prod.','Shared Date'],axis=1)
sk_unused = sk_unused.rename(columns={'_merge': 'UNIQUE'})

bc_unused = bc_unused.drop(['Dest._x','Dest._y','Status_x','Status_y','Market','Prod.','Shared Date'],axis=1)
bc_unused = bc_unused.rename(columns={'_merge': 'UNIQUE'})




Take a look at the dimensions of each dataframe. We can see some are uneven in the tuples provided.

In [41]:
print(ab_merge.shape,' ',ttab_dest.shape,' ',ttab_load.shape)
print(sk_merge.shape,' ',ttsk_dest.shape,' ',ttsk_load.shape)
print(bc_merge.shape,' ',ttbc_dest.shape,' ',ttbc_load.shape)
print(df.shape)

(57, 19)   (37, 12)   (37, 12)
(0, 19)   (4, 12)   (33, 12)
(141, 19)   (44, 12)   (60, 12)
(3813, 12)


In [42]:
# create a excel writer object
with pd.ExcelWriter("Merged_transfers"+".xlsx",engine='xlsxwriter') as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    bc_merge.to_excel(writer, sheet_name="BC Merge", index=True)
    ab_merge.to_excel(writer, sheet_name="AB Merge", index=True)
    sk_merge.to_excel(writer, sheet_name="SK Merge", index=True)

    ttbc_dest.to_excel(writer, sheet_name="TTBC dest", index=True)
    ttbc_load.to_excel(writer, sheet_name="TTBC load", index=True)

    ttab_dest.to_excel(writer, sheet_name="TTAB dest", index=True)
    ttab_load.to_excel(writer, sheet_name="TTAB load", index=True)

    ttsk_dest.to_excel(writer, sheet_name="TTSK dest", index=True)
    ttsk_load.to_excel(writer, sheet_name="TTSK load", index=True)

    bc_unused.to_excel(writer, sheet_name="BC Unmerged", index=True)
    ab_unused.to_excel(writer, sheet_name="AB Unmerged", index=True)
    sk_unused.to_excel(writer, sheet_name="SK Unmerged", index=True)

Generated output to file would be:

In [43]:
bc_merge

,Unnamed: 0_x,LOAD TMW#,Status_x,Started_x,Completed_x,Load_x,Dest._x,Prod.,Volume,Weight,Market,Shared Date,Unnamed: 0_y,DEST TMW#,Status_y,Started_y,Completed_y,Load_y,Dest._y
0,5530,133784,CMP,May,May,TTBC,ACCARM,1360X00,29000.0,39000.0,DUST,May,5517,133714,CMP,May,May,TIGKAM02,TTBC
1,5530,133784,CMP,May,May,TTBC,ACCARM,1360X00,29000.0,39000.0,DUST,May,5703,133925,CMP,May,May,TIGKAM02,TTBC
2,5530,133784,CMP,May,May,TTBC,ACCARM,1360X00,29000.0,39000.0,DUST,May,5716,133961,CMP,May,May,TIGKAM,TTBC
3,5530,133784,CMP,May,May,TTBC,ACCARM,1360X00,29000.0,39000.0,DUST,May,5770,134019,CMP,May,May,TIGKAM,TTBC
4,5530,133784,CMP,May,May,TTBC,ACCARM,1360X00,29000.0,39000.0,DUST,May,5981,134242,CMP,May,May,TIGKAM,TTBC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,8006,136800,CMP,Jun,Jun,TTBC,PREKIT,1360X00,30000.0,40000.0,DUST,Jun,7634,135953,CMP,Jun,Jun,TIGKAM,TTBC
137,8006,136800,CMP,Jun,Jun,TTBC,PREKIT,1360X00,30000.0,40000.0,DUST,Jun,8044,136325,CMP,Jun,Jun,TIGDAW,TTBC
138,8240,136947,CMP,Jun,Jun,TTBC,CONCHE,1360X00,26000.0,35000.0,DUST,Jun,7589,136019,CMP,Jun,Jun,TIGMIT,TTBC
139,8240,136947,CMP,Jun,Jun,TTBC,CONCHE,1360X00,26000.0,35000.0,DUST,Jun,7698,136166,CMP,Jun,Jun,TIGSPI,TTBC


In [44]:
ab_merge

,Unnamed: 0_x,LOAD TMW#,Status_x,Started_x,Completed_x,Load_x,Dest._x,Prod.,Volume,Weight,Market,Shared Date,Unnamed: 0_y,DEST TMW#,Status_y,Started_y,Completed_y,Load_y,Dest._y
0,5789,134041,CMP,May,May,TTAB,HILHIG,1360X00,8000.0,11000.0,DUST,May,5792,134018,CMP,May,May,TIGMIT,TTAB
1,6011,134288,CMP,May,May,TTAB,MACFOR,1360X00,28000.0,38000.0,DUST,May,5961,134218,CMP,May,May,TIGMIT,TTAB
2,6011,134288,CMP,May,May,TTAB,MACFOR,1360X00,28000.0,38000.0,DUST,May,7142,135310,CMP,May,May,TIGMIT,TTAB
3,6566,134738,CMP,May,May,TTAB,MCGSAS,1360X00,28000.0,38000.0,DUST,May,5961,134218,CMP,May,May,TIGMIT,TTAB
4,6566,134738,CMP,May,May,TTAB,MCGSAS,1360X00,28000.0,38000.0,DUST,May,7142,135310,CMP,May,May,TIGMIT,TTAB
5,6567,134739,CMP,May,May,TTAB,MCGSAS,1360X00,28000.0,38000.0,DUST,May,5961,134218,CMP,May,May,TIGMIT,TTAB
6,6567,134739,CMP,May,May,TTAB,MCGSAS,1360X00,28000.0,38000.0,DUST,May,7142,135310,CMP,May,May,TIGMIT,TTAB
7,6176,134497,CMP,May,May,TTAB,MACZAM,1360X00,26000.0,36000.0,DUST,May,6043,134365,CMP,May,May,TIGMIT,TTAB
8,6176,134497,CMP,May,May,TTAB,MACZAM,1360X00,26000.0,36000.0,DUST,May,6222,134500,CMP,May,May,TIGMIT,TTAB
9,6176,134497,CMP,May,May,TTAB,MACZAM,1360X00,26000.0,36000.0,DUST,May,7318,135585,CMP,May,May,TIGMIT,TTAB


In [45]:
sk_merge

,Unnamed: 0_x,LOAD TMW#,Status_x,Started_x,Completed_x,Load_x,Dest._x,Prod.,Volume,Weight,Market,Shared Date,Unnamed: 0_y,DEST TMW#,Status_y,Started_y,Completed_y,Load_y,Dest._y


In [46]:
bc_unused

,Unnamed: 0_x,TMW#_x,Started_x,Completed_x,Load_x,Volume,Weight,Unnamed: 0_y,TMW#_y,Started_y,Completed_y,Load_y,UNIQUE
0,5268.0,133486.0,Apr,Apr,TTBC,22000.0,31000.0,NaN,NaN,NaN,NaN,NaN,left_only
1,5272.0,133487.0,Apr,Apr,TTBC,6000.0,9000.0,NaN,NaN,NaN,NaN,NaN,left_only
2,5456.0,133684.0,May,May,TTBC,7000.0,9000.0,NaN,NaN,NaN,NaN,NaN,left_only
3,5481.0,133664.0,May,May,TTBC,22000.0,30000.0,NaN,NaN,NaN,NaN,NaN,left_only
4,6705.0,135025.0,May,May,TTBC,22000.0,30000.0,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,NaN,NaN,NaN,NaN,NaN,30000.0,41000.0,5574.0,133791.0,May,May,TIGKAM02,right_only
166,NaN,NaN,NaN,NaN,NaN,25000.0,35000.0,7069.0,135412.0,May,May,TIGMIT,right_only
167,NaN,NaN,NaN,NaN,NaN,27000.0,36000.0,7604.0,136020.0,Jun,Jun,TIGMIT,right_only
168,NaN,NaN,NaN,NaN,NaN,29000.0,39000.0,7753.0,136158.0,Jun,Jun,TIGMIT,right_only


In [47]:
ab_unused

,Unnamed: 0_x,TMW#_x,Started_x,Completed_x,Load_x,Volume,Weight,Unnamed: 0_y,TMW#_y,Started_y,Completed_y,Load_y,UNIQUE
0,5789.0,134041.0,May,May,TTAB,8000.0,11000.0,5792.0,134018.0,May,May,TIGMIT,both
1,6011.0,134288.0,May,May,TTAB,28000.0,38000.0,5961.0,134218.0,May,May,TIGMIT,both
2,6011.0,134288.0,May,May,TTAB,28000.0,38000.0,7142.0,135310.0,May,May,TIGMIT,both
3,6566.0,134738.0,May,May,TTAB,28000.0,38000.0,5961.0,134218.0,May,May,TIGMIT,both
4,6566.0,134738.0,May,May,TTAB,28000.0,38000.0,7142.0,135310.0,May,May,TIGMIT,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,NaN,NaN,NaN,NaN,NaN,27000.0,36000.0,7433.0,135690.0,May,May,TIGMIT,right_only
72,NaN,NaN,NaN,NaN,NaN,21000.0,28000.0,8011.0,136442.0,Jun,Jun,TIGMIT,right_only
73,NaN,NaN,NaN,NaN,NaN,0.0,0.0,8153.0,136703.0,Jun,Jun,TIGMIT,right_only
74,NaN,NaN,NaN,NaN,NaN,26000.0,36000.0,8292.0,137035.0,Jun,Jun,TIGMIT,right_only


In [48]:
sk_unused

,Unnamed: 0_x,TMW#_x,Started_x,Completed_x,Load_x,Volume,Weight,Unnamed: 0_y,TMW#_y,Started_y,Completed_y,Load_y,UNIQUE
0,7010.0,135280.0,May,May,TTSK,18000.0,24000.0,NaN,NaN,NaN,NaN,NaN,left_only
1,7015.0,135339.0,May,May,TTSK,11000.0,15000.0,NaN,NaN,NaN,NaN,NaN,left_only
2,7026.0,135340.0,May,May,TTSK,29000.0,39000.0,NaN,NaN,NaN,NaN,NaN,left_only
3,7444.0,135823.0,May,May,TTSK,27000.0,37000.0,NaN,NaN,NaN,NaN,NaN,left_only
4,7829.0,136301.0,Jun,Jun,TTSK,23000.0,31000.0,NaN,NaN,NaN,NaN,NaN,left_only
5,7937.0,136302.0,Jun,Jun,TTSK,23000.0,31000.0,NaN,NaN,NaN,NaN,NaN,left_only
6,7968.0,136305.0,Jun,Jun,TTSK,23000.0,31000.0,NaN,NaN,NaN,NaN,NaN,left_only
7,7972.0,136307.0,Jun,Jun,TTSK,23000.0,31000.0,NaN,NaN,NaN,NaN,NaN,left_only
8,8030.0,136310.0,Jun,Jun,TTSK,23000.0,31000.0,NaN,NaN,NaN,NaN,NaN,left_only
9,8039.0,136311.0,Jun,Jun,TTSK,23000.0,31000.0,NaN,NaN,NaN,NaN,NaN,left_only
